In [1]:
import numpy as np
import pandas as pd
import pprint as pp
import datetime as dt
import nltk
from nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

In [2]:
pd.set_option('display.max_colwidth', None)

nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/gilmar/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /Users/gilmar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gilmar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Load Datasets

Data Sources:
- [Elon Musk Tweets](https://www.kaggle.com/datasets/aryansingh0909/elon-musk-tweets-updated-daily/)
- [Dogecoin Historical Data](https://www.kaggle.com/datasets/dhruvildave/dogecoin-historical-data)
- [Bitcoin Price Dataset (2017-2023)](https://www.kaggle.com/datasets/jkraak/bitcoin-price-dataset)

In [3]:
def to_snake_case(value):
    return "_".join(value.lower().split())

In [4]:
def inspect_df(df, txt='Dataframe Information', date_col='date', col_prefix=None, sep_size=80):
    print('=' * sep_size)
    print(txt)
    print('=' * sep_size)
    date_range_counts_label = 'Dates Range and Row Count'
    print(date_range_counts_label)
    print('-' * len(date_range_counts_label))
    date_col = col_prefix + date_col if col_prefix else date_col
    pp.pprint(df[date_col].agg(['min','max','count']))
    print('-' * sep_size)
    pp.pprint(df.info())

In [5]:
tweets_df = pd.read_csv('data/elonmusk.csv')
dogecoin_df = pd.read_csv('data/DOGE-USD.csv')
bitcoin_df = pd.read_csv('data/bitcoin_2017_to_2023.csv')

Clean and prefix columns names

In [6]:
tweets_df.columns = [to_snake_case(c) for c in tweets_df.columns]
dogecoin_df.columns = [('dogecoin_' + to_snake_case(c)) for c in dogecoin_df.columns]
bitcoin_df.columns = [('bitcoin_' + to_snake_case(c)) for c in bitcoin_df.columns]

In [7]:
inspect_df(tweets_df, txt='Elon Musk Tweets - Original Dataset', date_col='datetime')
inspect_df(dogecoin_df, txt='Dogecoin - Original Dataset', col_prefix='dogecoin_')
inspect_df(bitcoin_df, txt='Bitcoin - Original Dataset', date_col='bitcoin_timestamp')

Elon Musk Tweets - Original Dataset
Dates Range and Row Count
-------------------------
min      2010-06-04 18:31:57+00:00
max      2023-06-29 16:20:19+00:00
count                        24450
Name: datetime, dtype: object
--------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24450 entries, 0 to 24449
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   datetime  24450 non-null  object
 1   tweet_id  24450 non-null  int64 
 2   text      24450 non-null  object
 3   username  24450 non-null  object
dtypes: int64(1), object(3)
memory usage: 764.2+ KB
None
Dogecoin - Original Dataset
Dates Range and Row Count
-------------------------
min      2017-11-09
max      2023-10-12
count          2164
Name: dogecoin_date, dtype: object
--------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeInde

## Normalise Dates

Convert all dates to datetime64[ns, UTC]

In [8]:
tweets_df['date'] = pd.DatetimeIndex(pd.to_datetime(tweets_df['datetime'])).normalize()
dogecoin_df['dogecoin_date'] = pd.to_datetime(dogecoin_df['dogecoin_date'], utc=True)
bitcoin_df['bitcoin_date'] = pd.DatetimeIndex(pd.to_datetime(bitcoin_df['bitcoin_timestamp'], utc=True)).normalize()

Trim datasets to have the same date range, ignoring time

In [9]:
tweets_df = tweets_df[tweets_df.date >= dogecoin_df.dogecoin_date.min()]
dogecoin_df = dogecoin_df[dogecoin_df.dogecoin_date <= tweets_df.date.max()]
bitcoin_df = bitcoin_df[(bitcoin_df.bitcoin_date  >= dogecoin_df.dogecoin_date.min()) \
                          & (bitcoin_df.bitcoin_date <= tweets_df.date.max())]

In [10]:
inspect_df(tweets_df, txt='Elon Musk Tweets')
inspect_df(dogecoin_df, txt='Dogecoin Data', col_prefix='dogecoin_')
inspect_df(bitcoin_df, txt='Bitcoin Data', col_prefix='bitcoin_')

Elon Musk Tweets
Dates Range and Row Count
-------------------------
min      2017-11-09 00:00:00+00:00
max      2023-06-29 00:00:00+00:00
count                        21525
Name: date, dtype: object
--------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 21525 entries, 0 to 21524
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype              
---  ------    --------------  -----              
 0   datetime  21525 non-null  object             
 1   tweet_id  21525 non-null  int64              
 2   text      21525 non-null  object             
 3   username  21525 non-null  object             
 4   date      21525 non-null  datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), int64(1), object(3)
memory usage: 1009.0+ KB
None
Dogecoin Data
Dates Range and Row Count
-------------------------
min      2017-11-09 00:00:00+00:00
max      2023-06-29 00:00:00+00:00
count                         2059
N

# Process Tweets

In [11]:
stopword = stopwords.words('english')
def tokenise(text):
    tokenizer = RegexpTokenizer(r'\w+')
    word_tokens = tokenizer.tokenize(text.lower())
    tokens = [word for word in word_tokens if word not in stopword]
    return tokens

In [12]:
dogecoin_keywords = ['dogecoin', 'dogecoins', 'doge']
cryptocoin_keywords = ['crypto', 'cryptocoins'\
                       , 'cryptocurrency', 'cryptocurrencies'\
                       , 'blockchain', 'blockchains'\
                       , 'bitcoin', 'bitcoins'\
                       , 'nft', 'nfts']
def mention(text, keywords):
    tokens = tokenise(text)
    mentions = [token for token in tokens if token in keywords]
    return 1 if mentions else 0

def mention_dogecoin(text):
    return mention(text, dogecoin_keywords)

def mention_crypto(text):
    return mention(text, dogecoin_keywords + cryptocoin_keywords)

In [13]:
def get_polarity_score(text):
    sia = SentimentIntensityAnalyzer()
    polarity_scores = sia.polarity_scores(text)
    return polarity_scores['compound']

In [14]:
def get_dogecoin_mention_polarity_score(text):
    score = 0.0
    if mention_dogecoin(text):
        score = get_polarity_score(text)
        #avoid mentions with 0.0 score
        if (score == 0.0):
            score = 0.01
    return score

In [15]:
tweets_df['mention_dogecoin'] = tweets_df.text.apply(mention_dogecoin)
tweets_df['mention_crypto'] = tweets_df.text.apply(mention_crypto)
tweets_df['dogecoin_mention_polarity_score'] = tweets_df.text.apply(get_dogecoin_mention_polarity_score)

In [16]:
print('Elon tweets mentioning Dogecoin: ' + str(len(tweets_df[tweets_df.mention_dogecoin == 1])))

Elon tweets mentioning Dogecoin: 84


## Daily Tweets

Sum mentions and polarity score by date

In [17]:
daily_tweets_df = tweets_df.groupby(['date'])\
    .agg({'mention_dogecoin':'sum'\
          , 'mention_crypto' : 'sum'\
          , 'dogecoin_mention_polarity_score':'sum'})\
    .reset_index()
daily_tweets_df = daily_tweets_df.rename(columns=\
                                         {'mention_dogecoin': 'dogecoin_mentions'\
                                         , 'mention_crypto': 'crypto_mentions'})

Assert no mention has polarity score zero

In [18]:
assert(len(daily_tweets_df[daily_tweets_df.dogecoin_mentions > 0]) == \
len(daily_tweets_df[(daily_tweets_df.dogecoin_mentions > 0) & (daily_tweets_df.dogecoin_mention_polarity_score != 0)]))

In [19]:
inspect_df(daily_tweets_df, txt='Daily Tweets')

Daily Tweets
Dates Range and Row Count
-------------------------
min      2017-11-09 00:00:00+00:00
max      2023-06-29 00:00:00+00:00
count                         1874
Name: date, dtype: object
--------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1874 entries, 0 to 1873
Data columns (total 4 columns):
 #   Column                           Non-Null Count  Dtype              
---  ------                           --------------  -----              
 0   date                             1874 non-null   datetime64[ns, UTC]
 1   dogecoin_mentions                1874 non-null   int64              
 2   crypto_mentions                  1874 non-null   int64              
 3   dogecoin_mention_polarity_score  1874 non-null   float64            
dtypes: datetime64[ns, UTC](1), float64(1), int64(2)
memory usage: 58.7 KB
None


# Prepare Bitcoin Data

In [20]:
bitcoin_df = bitcoin_df.sort_values('bitcoin_timestamp')

## Daily Bitcoin

### Get last close value for each day

In [21]:
daily_bitcoin_close_df = bitcoin_df\
            .groupby('bitcoin_date')\
            .tail(1)\
            [['bitcoin_date', 'bitcoin_close']]

In [22]:
inspect_df(daily_bitcoin_close_df, 'Daily last bitcoin close', col_prefix="bitcoin_")

Daily last bitcoin close
Dates Range and Row Count
-------------------------
min      2017-11-09 00:00:00+00:00
max      2023-06-29 00:00:00+00:00
count                         2059
Name: bitcoin_date, dtype: object
--------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2059 entries, 3004259 to 46880
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   bitcoin_date   2059 non-null   datetime64[ns, UTC]
 1   bitcoin_close  2059 non-null   float64            
dtypes: datetime64[ns, UTC](1), float64(1)
memory usage: 48.3 KB
None


### Sum bitcoin volume to get total for each day

In [23]:
daily_bitcoin_volume_df = bitcoin_df[['bitcoin_date','bitcoin_volume']]\
            .groupby('bitcoin_date')\
            .sum()\
            .reset_index()

In [24]:
inspect_df(daily_bitcoin_volume_df, 'Daily bitcoin total volume', col_prefix="bitcoin_")

Daily bitcoin total volume
Dates Range and Row Count
-------------------------
min      2017-11-09 00:00:00+00:00
max      2023-06-29 00:00:00+00:00
count                         2059
Name: bitcoin_date, dtype: object
--------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2059 entries, 0 to 2058
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   bitcoin_date    2059 non-null   datetime64[ns, UTC]
 1   bitcoin_volume  2059 non-null   float64            
dtypes: datetime64[ns, UTC](1), float64(1)
memory usage: 32.3 KB
None


In [25]:
daily_bitcoin_df = daily_bitcoin_close_df.merge(daily_bitcoin_volume_df)

In [26]:
inspect_df(daily_bitcoin_df, 'Daily bitcoin total volume', col_prefix="bitcoin_")

Daily bitcoin total volume
Dates Range and Row Count
-------------------------
min      2017-11-09 00:00:00+00:00
max      2023-06-29 00:00:00+00:00
count                         2059
Name: bitcoin_date, dtype: object
--------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2059 entries, 0 to 2058
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   bitcoin_date    2059 non-null   datetime64[ns, UTC]
 1   bitcoin_close   2059 non-null   float64            
 2   bitcoin_volume  2059 non-null   float64            
dtypes: datetime64[ns, UTC](1), float64(2)
memory usage: 64.3 KB
None


# Prepare Final Dataset

## Elon's Tweets + Dogecoin Data

In [27]:
daily_dogecoin_df = dogecoin_df[['dogecoin_date','dogecoin_close', 'dogecoin_volume']]
tweets_doge_df = daily_dogecoin_df.set_index('dogecoin_date')\
    .join(daily_tweets_df.set_index('date'), on='dogecoin_date')\
    .reset_index()
# no dogecoin mentions and polarity score for days elon didn't tweet
tweets_doge_df = tweets_doge_df.fillna(0.0)
tweets_doge_df['date'] = tweets_doge_df['dogecoin_date']
tweets_doge_df = tweets_doge_df.drop('dogecoin_date',axis=1)

Check we have all days

In [28]:
init_day = tweets_df['date'].min()
end_day = tweets_df['date'].max()
time_delta = end_day - init_day
assert(len(tweets_doge_df) == (time_delta.days + 1))

In [29]:
inspect_df(tweets_doge_df, txt='Daily Tweets + Dogecoin')

Daily Tweets + Dogecoin
Dates Range and Row Count
-------------------------
min      2017-11-09 00:00:00+00:00
max      2023-06-29 00:00:00+00:00
count                         2059
Name: date, dtype: object
--------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2059 entries, 0 to 2058
Data columns (total 6 columns):
 #   Column                           Non-Null Count  Dtype              
---  ------                           --------------  -----              
 0   dogecoin_close                   2059 non-null   float64            
 1   dogecoin_volume                  2059 non-null   int64              
 2   dogecoin_mentions                2059 non-null   float64            
 3   crypto_mentions                  2059 non-null   float64            
 4   dogecoin_mention_polarity_score  2059 non-null   float64            
 5   date                             2059 non-null   datetime64[ns, UTC]
dtypes: dateti

## Elon's Tweets + Dogecoin + Bitcoin Data

In [30]:
tweets_doge_bitcoin_df = daily_bitcoin_df.merge(tweets_doge_df, left_on='bitcoin_date', right_on='date')
tweets_doge_bitcoin_df = tweets_doge_bitcoin_df.drop('bitcoin_date', axis=1)
tweets_doge_bitcoin_df['date'] = tweets_doge_bitcoin_df['date'].dt.date

In [31]:
## check we still have all days
assert(len(tweets_doge_bitcoin_df) == (time_delta.days + 1))

In [32]:
inspect_df(tweets_doge_bitcoin_df, 'Tweets + Dogecoin + Bitcoin')

Tweets + Dogecoin + Bitcoin
Dates Range and Row Count
-------------------------
min      2017-11-09
max      2023-06-29
count          2059
Name: date, dtype: object
--------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2059 entries, 0 to 2058
Data columns (total 8 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   bitcoin_close                    2059 non-null   float64
 1   bitcoin_volume                   2059 non-null   float64
 2   dogecoin_close                   2059 non-null   float64
 3   dogecoin_volume                  2059 non-null   int64  
 4   dogecoin_mentions                2059 non-null   float64
 5   crypto_mentions                  2059 non-null   float64
 6   dogecoin_mention_polarity_score  2059 non-null   float64
 7   date                             2059 non-null   object 
dtypes: float64(6), int64(1

In [33]:
tweets_doge_bitcoin_df

,bitcoin_close,bitcoin_volume,dogecoin_close,dogecoin_volume,dogecoin_mentions,crypto_mentions,dogecoin_mention_polarity_score,date
0,7079.99,1913.398308,0.001415,6259550,0.0,0.0,0.0,2017-11-09
1,6506.98,3254.704105,0.001163,4246520,0.0,0.0,0.0,2017-11-10
2,6245.05,2754.156861,0.001201,2231080,0.0,0.0,0.0,2017-11-11
3,5811.03,4968.483069,0.001038,3288960,0.0,0.0,0.0,2017-11-12
4,6465.99,2621.243039,0.001211,2481270,0.0,0.0,0.0,2017-11-13
...,...,...,...,...,...,...,...,...
2054,30462.66,30223.448010,0.066764,191028870,0.0,0.0,0.0,2023-06-25
2055,30267.99,45180.414890,0.064749,266917426,0.0,0.0,0.0,2023-06-26
2056,30692.44,42699.641570,0.065545,212409649,0.0,0.0,0.0,2023-06-27
2057,30077.41,40463.519370,0.062602,274764095,0.0,0.0,0.0,2023-06-28


# Save Data as CSV

In [34]:
tweets_doge_bitcoin_df.set_index('date').to_csv('data/tweets_doge_bitcoin.csv')